In [ ]:
# Xgboost
'''
XGBoost는 Gradient Boosting 알고리즘을 분산환경에서도 실행할 수 있도록
구현해놓은 라이브러리이다
XGBoost는 여러개의 Decision Tree를 조합해서 사용하는 ensemble 알고리즘이다

boosting(부스팅)
- 라운드가 지날수록 모델의 에러를 줄이는 과정
- 어떤 모델이 유효한지, 적절한지를 찾아내는 과정
- 약한 예측 모형들을 결합해 강한 예측 모형을 만드는 알고리즘
- 무작위성이 없으며 강력한 사전 가지치기 사용

결측치를 내부적으로 처리해준다
'''

In [ ]:
# parameter
'''
booster : 부스터 구조 선택
- gbtree = Gradient  Boosted Decision trees
- gblinear = Gradient Boosted linear
- dart = Dropout Regression Trees

n_estimators : 결정 트리의 개수
n_jobs : 동시에 사용할 쓰레드 개수(default는 가능한 많이)
num_feature : feature 차원의 숫자를 정해야 할 경우 사용 (default는 가능한 많이)
num_boost_rounds : boosting 라운드를 결정한다
                   랜덤하게 생성되는 모델이므로 적당히 큰 수가 좋다
nrounds : 부스팅 반복 횟수

colsample_bytree : 컬럼의 샘플링 비율
subsample : 약한 모델이 학습에 사용하는 데이터 샘플링 비율

eta : learning rate로 boosting step마다 weight를 줘서 부스팅 과정에
      과적합(overfitting)이 일어나지 않도록 한다
gamma : information gain에서 r로 표현한 값. 이 값이 커지면 트리 깊이가 줄어들어
        보수적인 모델이 된다
max_depth : 한 트리의 maximum depth. default는 6이고
            이 경우 리프 노드의 개수는 최대 2^6개
lambda(l2 reg-form) : l2 regularization form weight로
                      숫자가 클수록 보수적인 모델이다
alpha(l1 reg-form) : l1 regularization form weight로
                     숫자가 클수록 보수적인 모델이다

objective : 목적함수로 reg:linear, binary:logistic, count:poisson 등 다양하다
eval_metric : 모델 평가 함수로 rmse, logloss, mae 등이 있다

early_stopping_rounds : 특정 횟수를 지나도 모델이 개선되지 않을 때
                        조기에 종료시킨다
'''

In [ ]:
# example
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [ ]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [ ]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

y = train[:,0]
y = np.reshape(y, (-1,1))
en = LabelEncoder()
y = en.fit_transform(y)

print(x.shape)
print(y.shape)

In [ ]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [ ]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

x_total = np.reshape(x_total, (-1, 784))
    
print(x_total.shape)
print(y_total.shape)

In [ ]:
pca_x = x_total.copy()
pca = PCA()
pca.fit(pca_x)
cumsum = np.cumsum(pca.explained_variance_ratio_)
n = np.argmax(cumsum >= 0.99) + 1
print(n)

def pca_data(data, n):
    pca = PCA(n_components=n)
    x = data.copy()
    w = pca.fit_transform(x)
    return w

x_pca = pca_data(x_total, n+4)
print(x_pca.shape)

In [ ]:
#x_train, x_val, y_train, y_val = train_test_split(x_pca, y_total,
#                                                  test_size=0.2, shuffle=True)

#print(x_train.shape)
#print(x_val.shape)
#print(y_train.shape)
#print(y_val.shape)

In [ ]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softmax')
xgb_params = {'max_depth':[3,5,7,9], 'n_estimators':[50,100,150,200],
              'learning_rate':[0.01,0.05,0.1,0.15,0.2],
              'subsample':[0.6,0.8,1.0],'min_child_weight':[1,5,10],
              'gamma':[0.5,1,1.5,2], 'max_delta_step':[1,3,5,7,9],
              'reg_alpha':[1e-5,1e-2,0.1,1,100]}
search = GridSearchCV(xgb, xgb_params, cv=5)
best_xgb = search.fit(x_pca, y_total, verbose=1)
best_params = search.best_params_

In [ ]:
print(best_xgb)
print(best_params)

In [ ]:
x_test = test[:,1:]
x_test = np.reshape(x_test, (-1, 784))
test_pca = pca_data(x_test, n+4)
print(test_pca.shape)

In [ ]:
pred = best_xgb.predict(test_pca)
pred

In [ ]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = np.argmax(best_model.predict(x_test), axis=1)
#submission.to_csv('data/submission_xgboost(0907).csv', index=False)